# Modèle de prédation de Lotka-Volterra

Les équations de Lotka-Volterra, également connues sous le nom de modèle proie-prédateur, sont un système d'équations différentielles non linéaires du premier ordre qui décrivent les interactions entre une population de proies et de prédateurs.

## Équations du modèle

Le modèle est défini par le système d'équations suivant :

$$
\begin{cases}
\frac{dx}{dt} = x(t) \left( \alpha - \beta y(t) \right) \\
\frac{dy}{dt} = y(t) \left( \delta x(t) - \gamma \right)
\end{cases}
$$

### Paramètres
- $t$ : temps
- $x(t)$ : population des proies au temps $t$
- $y(t)$ : population des prédateurs au temps $t$
- $\alpha$ : taux de reproduction des proies
- $\beta$ : taux de mortalité des proies dû aux prédateurs
- $\delta$ : taux de reproduction des prédateurs en fonction des proies consommées
- $\gamma$ : taux de mortalité des prédateurs

### Signification physique des équations
#### Équation des proies
$$
\frac{dx}{dt} = \alpha x(t) - \beta x(t) y(t)
$$
Les proies se reproduisent exponentiellement sans prédation ($\alpha x(t)$) et subissent une mortalité due aux rencontres avec les prédateurs ($\beta x(t) y(t)$).

#### Équation des prédateurs
$$
\frac{dy}{dt} = \delta x(t) y(t) - \gamma y(t)
$$
Les prédateurs se reproduisent en fonction de la disponibilité des proies ($\delta x(t) y(t)$) et subissent une mortalité naturelle ($\gamma y(t)$).

## Solutions et dynamiques

### Comportement global et intégrale première
Pour des conditions initiales $x(t_0) > 0$ et $y(t_0) > 0$, la solution est définie pour tout $t \in \mathbb{R}$ et vérifie $x(t) > 0$ et $y(t) > 0$.
L'intégrale première du mouvement est donnée par :
$$
F(x, y) = \beta y + \delta x - \alpha \ln y - \gamma \ln x
$$
Les solutions sont périodiques et fermées, oscillant de manière cyclique.

### Moyennes des solutions
Les moyennes des populations sur une période $T$ sont :
$$
\langle x \rangle = \frac{\gamma}{\delta}, \quad \langle y \rangle = \frac{\alpha}{\beta}
$$
En incluant des termes de disparition supplémentaires ($\varepsilon x(t)$ et $\phi y(t)$), les moyennes deviennent :
$$
\langle x \rangle = \frac{\gamma + \phi}{\delta}, \quad \langle y \rangle = \frac{\alpha - \varepsilon}{\beta}
$$

## Points d'équilibre

Les points d'équilibre sont obtenus lorsque :
$$
\begin{cases}
x(t) (\alpha - \beta y(t)) = 0 \\
-y(t) (\gamma - \delta x(t)) = 0
\end{cases}
$$
Les solutions sont :
1. $x = 0$ et $y = 0$ : extinction des deux espèces.
2. $x = \frac{\gamma}{\delta}$ et $y = \frac{\alpha}{\beta}$ : équilibre stable des populations.

### Stabilité des points fixes

- **Premier point fixe** $(0, 0)$ :
  $$
  J(0, 0) = \begin{bmatrix} \alpha & 0 \\ 0 & -\gamma \end{bmatrix}
  $$
  Les valeurs propres sont de signes opposés ($\lambda_1 = \alpha$, $\lambda_2 = -\gamma$), ce qui indique un point selle (instable).

- **Second point fixe** $\left( \frac{\gamma}{\delta}, \frac{\alpha}{\beta} \right)$ :
  $$
  J\left( \frac{\gamma}{\delta}, \frac{\alpha}{\beta} \right) = \begin{bmatrix} 0 & -\frac{\beta \gamma}{\delta} \\ \frac{\alpha \delta}{\beta} & 0 \end{bmatrix}
  $$
  Les valeurs propres sont imaginaires pures ($\lambda_1 = i \sqrt{\alpha \gamma}$, $\lambda_2 = -i \sqrt{\alpha \gamma}$), indiquant un centre stable avec oscillations périodiques des populations.

## Dynamique du système

Les populations des proies et prédateurs oscillent de manière cyclique. Les prédateurs prospèrent quand les proies sont nombreuses, mais leur population décroît après avoir épuisé les ressources. Les proies se multiplient alors, ce qui permet aux prédateurs de se rétablir, et ainsi de suite.


### Import des librairies

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
from scipy.integrate import odeint
from ipywidgets import interactive, VBox, Button, widgets
from IPython.display import display
import ipywidgets as widgets

In [78]:
# Fonction du modèle de Lotka-Volterra
def lotka_volterra(z, t, alpha, beta, delta, gamma):
    x, y = z
    dxdt = x * (alpha - beta * y)
    dydt = y * (delta * x - gamma)
    return [dxdt, dydt]

# Fonction pour tracer le graphe et sauvegarder
def plot_model(x_0, y_0, alpha, beta, delta, gamma):
    T_span = (0, 100)
    T = np.linspace(*T_span, 1000)
    z_0 = [x_0, y_0]
    sol = odeint(lotka_volterra, z_0, T, args=(alpha, beta, delta, gamma))

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(T, sol[:, 0], label='Population proies')
    ax.plot(T, sol[:, 1], label='Population prédateurs')
    ax.set_xlabel('Années')
    ax.set_ylabel('Population')
    ax.set_title(f'Modèle de Lotka-Volterra sur {round(np.max(T))} ans')
    ax.legend()
    ax.grid(True)
    
    # Bouton pour sauvegarder le graphique
    save_button = Button(description="Save Graph")
    
    # Fonction de sauvegarde
    def save_graph(_):
        fig.savefig('modele_lotka_volterra.png')
        print("Graphique enregistré sous 'modele_lotka_volterra.png'")

    # Lier le bouton à la fonction de sauvegarde
    save_button.on_click(save_graph)
    
    display(save_button)
    plt.show()

# Curseurs interactifs et bouton de sauvegarde
interactive_plot = interactive(plot_model,
                               x_0=widgets.IntSlider(min=10, max=500, step=10, value=100, description='Proies initiales'),
                               y_0=widgets.IntSlider(min=10, max=500, step=10, value=50, description='Prédateurs initiaux'),
                               alpha=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.1, description='Taux de reproduction proies (α)'),
                               beta=widgets.FloatSlider(min=0.01, max=0.1, step=0.01, value=0.02, description='Mortalité due aux prédateurs (β)'),
                               delta=widgets.FloatSlider(min=0.005, max=0.05, step=0.005, value=0.01, description='Reproduction prédateurs (δ)'),
                               gamma=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.4, description='Mortalité prédateurs (γ)'))

display(VBox([interactive_plot]))


### Sensibilité des Modèles Proie-Prédateur aux Échelles des Populations

Le modèle de Lotka-Volterra présente une sensibilité marquée aux échelles des populations de départ. Même avec des taux de reproduction et de mortalité constants, les dynamiques du système changent radicalement en fonction de la taille initiale des populations. Cette sensibilité est liée aux interactions non linéaires entre les proies et les prédateurs, modélisées par le produit des deux populations.

**Exemple Illustratif**
Pour illustrer ce phénomène, considérons deux cas :
    - Populations de départ faibles : 10 proies et 5 prédateurs
    - item Populations de départ élevées : 100 proies et 50 prédateurs

Dans ces deux cas, les paramètres sont identiques : $\alpha = 1.1$, $\beta = 0.4$, $\delta = 0.1$, et $\gamma = 0.4$.

#### Cas 1 : Faibles populations
Avec ces valeurs initiales, le terme $x' = x(\alpha - \beta y)$ pour les proies conduit à une diminution modérée de 9 unités, et le terme $y' = y(\delta x - \gamma)$ pour les prédateurs conduit à une augmentation de 3 unités.

#### Cas 2 : Grandes populations
Ici, le terme $x' = x(\alpha - \beta y)$ pour les proies entraîne une chute drastique de 1990 unités, tandis que $y' = y(\delta x - \gamma)$ conduit à une augmentation de 480 unités pour les prédateurs.

Les résultats montrent que, bien que les ratios et les paramètres soient identiques, les grandes populations subissent des variations extrêmes dès les premières étapes.

**Explication du Phénomène**
Ce comportement s'explique par l'amplification des interactions absolues entre les proies et les prédateurs :

- Le produit $x \times y$ qui représente les interactions, augmente proportionnellement aux valeurs de $x$ et $y$. Cela entraîne une intensité accrue des rencontres proies-prédateurs pour les grandes populations.
    
- Dans des systèmes avec de petites populations, la régulation est progressive, permettant l’apparition d’oscillations. En revanche, pour des populations élevées, l’intensité des interactions cause des changements rapides qui peuvent conduire à l'extinction ou à des oscillations non stables.

#### Recommandations pour la Modélisation
Lors de la modélisation de systèmes proie-prédateur, il est essentiel de prendre en compte cette sensibilité aux échelles des populations :
- Ajustement des Taux : Pour des populations élevées, réduire les taux de reproduction et de mortalité $\alpha$, $\beta$, $\delta$, et $\gamma$ afin de modérer les interactions.
- Validation des Paramètres : Avant de lancer une simulation, tester différents taux pour observer les comportements. Cela permet de vérifier si le système évolue avec des oscillations stables ou si des extinctions rapides sont probables.
- Surveillance des Interactions Absolues: En utilisant des valeurs élevées, surveiller les produits d'interaction pour anticiper des variations extrêmes dans les populations.

Si l'interaction $x \times y$ devient trop grande parce que les populations initiales sont élevées, il est nécessaire de réduire les paramètres $\beta$ et $\delta$ pour modérer l'impact de cette interaction sur le système.

### Pourquoi Diminuer $\beta$ et $\delta$ ?
- **$\beta$** : Ce paramètre détermine l'impact des prédateurs sur les proies. Si $\beta$ est élevé et que $x$ et $y$ sont grands, cela peut entraîner une diminution rapide de la population de proies. En réduisant $\beta$, on limite la vitesse à laquelle les prédateurs consomment les proies.
  
- **$\delta$** : Ce paramètre mesure l'impact des proies sur la reproduction des prédateurs. Si $\delta$ est élevé et que $x$ et $y$ sont grands, cela peut entraîner une augmentation rapide de la population de prédateurs. En réduisant \$delta$, on freine la croissance des prédateurs due à l'abondance des proies.

### Contrebalancer l'Interaction
En ajustant $\beta$ et $\delta$ à la baisse, on peut compenser l’effet amplifié de $x \times y$  sur les populations, permettant d’obtenir des oscillations plus contrôlées et de prévenir des extinctions ou explosions de population. C'est une façon de stabiliser le système lorsque les valeurs de $x$ et $y$ sont élevées.


### Le modèle de Rosenzweig-MacArthur

Le modèle de Rosenzweig-MacArthur est une variante du modèle Lotka-Volterra, qui inclut un taux d'ingestion saturé pour les prédateurs. L'expression "ingestion saturée pour les prédateurs" fait référence au fait que le taux de consommation de proies par les prédateurs n'augmente pas indéfiniment avec la densité de proies. En effet, il atteint un niveau maximal ou une saturation.

Plus précisément, dans la nature les prédateurs ont une capacité limitée à ingérer des proies, même si la quantité de proies est abondante. Par exemple, un prédateur ne peut pas manger sans cesse ; au bout d'un moment, il devient rassasié ou est limité par le temps nécessaire pour chasser et digérer. On peut modélisée cette limite par une fonction qui augmente avec la densité de proies mais qui tend asymptotiquement vers un maximum.

Pour le modèle de Rosenzweig-MacArthur, l'ingestion est représentée par une fonction de type Holling Type II :
$$ Taux \,d'ingestion = \frac{ax}{1 + hx}$$

- $x$ : Densité des proies.
- $a$ : Taux d'attaque (c'est-à-dire à quelle vitesse le prédateur capture les proies).
- $h$ : Temps de manipulation, soit le temps que prend un prédateur pour capturer et consommer une proie.

Lorsque la densité de proies est faible, le terme $1 + hx$ est proche de 1, et le taux d'ingestion est environ linéaire avec $x$. Mais à mesure que $x$ augmente, le dénominateur augmente également, ce qui fait que le taux d'ingestion se rapproche d'une valeur maximale $\frac{a}{h}$.

Le modèle se présente comme suit :
$$ 
\begin{cases}
\frac{dx}{dt} = r * x(t) \left( 1 - \frac{x(t)}{K} \right) - \frac{ax(t)y(t)}{1 + hx(t)} \\
\frac{dy}{dt} = e * \frac{ax(t)y(t)}{1 + hx(t)} - d*y(t)
\end{cases}
$$
où

- $r$ est le taux de croissance intrinsèque des proies:
  - Elle représente le taux de croissance maximal de la population de proies en l'absence de prédateurs.

- $K$ est la capacité limite de l'environnement pour les proies :
  - Elle représente la capacité maximale que l’environnement peut supporter pour la population de proies. 

- $a$ est le taux d'attaque des prédateurs:
  - Ce paramètre indique la vitesse à laquelle les prédateurs capturent les proies. Un taux d'attaque plus élevé signifie que les prédateurs sont plus efficaces pour capturer les proies.

- $h$ est le temps de manipulation des proies:
  - $h$ représente le temps nécessaire pour qu'un prédateur capture, consomme, et digère une proie. Ce paramètre crée la saturation dans le taux d’ingestion des prédateurs.

- $e$ mesure l'efficacité de conversion des proies en prédateurs:
  - Cette constante représente la fraction de proies consommées qui est convertie en nouvelle population de prédateurs. En d’autres termes, c'est le taux auquel la nourriture (proies) se traduit en croissance de la population des prédateurs.

- $d$ est le taux de mortalité des prédateurs:
  - Elle est le taux de mortalité des prédateurs, indiquant le taux auquel la population des prédateurs diminue naturellement en l’absence de nourriture.


In [79]:
# Fonction du modèle Rosenzweig-MacArthur
def rosenzweig_macarthur(z, t, r, K, a, h, e, d):
    x, y = z
    dxdt = r * x * (1 - x / K) - (a * x * y) / (1 + h * x)
    dydt = e * (a * x * y / (1 + h * x)) - d * y
    return [dxdt, dydt]

# Fonction pour tracer le graphe et sauvegarder
def plot_model(x_0, y_0, r, K, a, h, e, d):
    T_span = (0, 100)
    T = np.linspace(*T_span, 1000)
    z_0 = [x_0, y_0]
    sol = odeint(rosenzweig_macarthur, z_0, T, args=(r, K, a, h, e, d))

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(T, sol[:, 0], label='Population proies')
    ax.plot(T, sol[:, 1], label='Population prédateurs')
    ax.set_xlabel('Années')
    ax.set_ylabel('Population')
    ax.set_title(f'Modèle de Rosenzweig-MacArthur sur {round(np.max(T))} ans')
    ax.legend()
    ax.grid(True)
    
    # Bouton pour sauvegarder le graphique
    save_button = Button(description="Save Graph")
    
    # Fonction de sauvegarde
    def save_graph(_):
        fig.savefig('rosenzweig_macarthur.png')
        print("Graphique enregistré sous 'rosenzweig_macarthur.png'")

    # Lier le bouton à la fonction de sauvegarde
    save_button.on_click(save_graph)
    
    display(save_button)
    plt.show()

# Curseurs interactifs et bouton de sauvegarde
interactive_plot = interactive(plot_model,
                               x_0=widgets.IntSlider(min=10, max=500, step=10, value=100, description='Proies initiales'),
                               y_0=widgets.IntSlider(min=10, max=500, step=10, value=50, description='Prédateurs initiaux'),
                               r=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.5, description='Croissance proies (r)'),
                               K=widgets.IntSlider(min=500, max=2000, step=100, value=1000, description='Capacité environnementale (K)'),
                               a=widgets.FloatSlider(min=0.01, max=0.1, step=0.01, value=0.03, description="Taux d'attaque (a)"),
                               h=widgets.FloatSlider(min=0.001, max=0.01, step=0.0005, value=0.0015, description='Temps de manipulation (h)'),
                               e=widgets.FloatSlider(min=0.1, max=0.5, step=0.05, value=0.15, description="Conversion proies en prédateurs (e)"),
                               d=widgets.FloatSlider(min=0.1, max=0.5, step=0.05, value=0.15, description="Mortalité prédateurs (d)"))

display(VBox([interactive_plot]))


#### Commentaire sur les paramètres :

##### 1. **Taux de croissance des proies, \( r \)**
   - **Explication** : Ce paramètre décrit la vitesse à laquelle la population de proies croît en l’absence de prédateurs, grâce aux ressources abondantes de l’environnement. En termes simples, un taux de croissance de 0,5 signifie que, si la population initiale est de 100, au bout d'une unité de temps (disons une année), la population aura augmenté de 50 %, soit 150.
   - **Formule** : Si la population initiale est \( x_0 \), après un temps \( t \), sans prédateurs, elle serait approximativement $( x_0 \times (1 + r)^t)$.  Par exemple, avec \( r = 0.5 \) pour un temps de 1, une population de 100 atteindrait 150.

##### 2. **Capacité de l’environnement, \( K \)**
   - **Explication** : C’est la population maximale que l’environnement peut soutenir. Lorsqu'une population atteint \( K \), la croissance ralentit car les ressources (comme la nourriture) se raréfient.

##### 3. **Taux d'attaque des prédateurs, \( a \)**
   - **Explication** : Il représente l’efficacité avec laquelle les prédateurs capturent les proies. Un taux de 0,1 signifie que chaque prédateur peut attraper 10 % de la population de proies disponible, par unité de temps.
   - **Intuition** : Si \( a \) est élevé, les prédateurs sont très efficaces et peuvent réduire rapidement la population de proies. À l’inverse, un taux faible implique que les prédateurs capturent moins de proies.

##### 4. **Temps de manipulation des proies, \( h \)**
   - **Explication** : Ce paramètre exprime le temps nécessaire pour qu’un prédateur manipule et consomme une proie. Un \( h \) de 0,1 signifie qu’il faut 0,1 unité de temps (disons 0,1 jour) pour qu’un prédateur manipule une proie avant de passer à une autre. Plus \( h \) est élevé, plus le prédateur met du temps à consommer une proie, ce qui ralentit la prédation.
   - **Intuition** : Si \( h \) est très petit (comme 0,001), les prédateurs peuvent consommer de nombreuses proies rapidement. À l’inverse, un \( h \) grand réduit la consommation de proies, ce qui peut diminuer la pression de prédation sur les proies.

#### 5. **Efficacité de conversion des proies en prédateurs, \( e \)**
   - **Explication** : C’est le pourcentage de proies consommées qui est converti en nouvelle croissance pour les prédateurs. Par exemple, \( e = 0.1 \) signifie que 10 % des proies consommées sont converties en nouveaux prédateurs. 
   - **Intuition** : Un \( e \) faible signifie que même si les prédateurs capturent beaucoup de proies, cela ne se traduit pas par une augmentation importante de leur population. Un \( e \) élevé, en revanche, mène à une croissance plus rapide des prédateurs.

##### 6. **Taux de mortalité des prédateurs, \( d \)**
   - **Explication** : Il s’agit de la proportion de la population de prédateurs qui disparaît (meurt) par unité de temps. Un \( d \) de 0,1 signifie qu’en l’absence de proies, 10 % des prédateurs meurent chaque unité de temps.

#### **REMARQUE:**  
Le taux de mortalité des prédateurs, l’efficacité de conversion des proies, et le temps de manipulation des proies — sont cruciaux car ils déterminent la capacité des prédateurs à survivre et à influencer les proies. En effet,

##### 1. **Taux de mortalité des prédateurs d**
   - **Impact** : Un taux de mortalité élevé signifie que les prédateurs ont du mal à survivre en l'absence de proies abondantes. Si ce taux est trop élevé, même un petit déclin dans la population de proies peut entraîner un déclin rapide des prédateurs, menant parfois à une extinction.

##### 2. **Efficacité de conversion des proies en prédateurs e**
   - **Impact** : Ce paramètre détermine combien de proies sont nécessaires pour soutenir une population de prédateurs. Si e est faible, les prédateurs ont besoin de consommer beaucoup de proies pour croître. Cela peut entraîner une situation où les proies sont trop consommées, suivie d’une chute des prédateurs lorsqu'ils manquent de proies. Un e trop élevé signifie que même une petite quantité de proies entraîne une forte augmentation de la population de prédateurs, ce qui peut mener à une oscillation excessive ou à une extinction de masse des proies.

##### 3. **Temps de manipulation des proies h**
   - **Impact** : h faible signifie que les prédateurs consomment les proies très rapidement. Ce qui peut entraîner une consommation excessive des proies, suivi d’un effondrement de leur population. Si h est élevé, les prédateurs ont moins d’impact immédiat sur les proies, ce qui peut stabiliser les oscillations. Un temps de manipulation très bas (par exemple 0.001) peut induire des oscillations rapides et des cycles d'extinctions rapides.

Ces paramètres influencent directement la **dynamique de feedback** dans le modèle. Une petite modification dans l'un d'eux peut faire basculer le système entre une situation où les deux populations oscillent de manière stable, et une situation où l'une des populations disparaît rapidement.

## Introduction à l’Équation de Fisher-KPP et aux Systèmes de Réaction-Diffusion

### 1. Système de réaction-diffusion
Les systèmes de réaction-diffusion sont des modèles mathématiques utilisés pour décrire des phénomènes où une réaction locale est couplée à un terme de diffusion. En chimie, cela correspond souvent à des réactions chimiques où les substances se transforment les unes dans les autres et se diffusent dans l'espace.

Mathématiquement, un système de réaction-diffusion peut être représenté par une équation aux dérivées partielles (EDP) semi-linéaire parabolique :
$$
\partial_t \mathbf{q} = \mathbf{D} \nabla^2 \mathbf{q} + \mathbf{R}(\mathbf{q})
$$
où $\mathbf{q}(x, t)$ est la fonction vecteur inconnue, $\mathbf{D}$ est une matrice de coefficients de diffusion, et $\mathbf{R}$ représente les réactions locales. Ces systèmes peuvent produire des comportements complexes tels que des ondes progressives ou des motifs auto-organisés.

### 2. Introduction à l’équation de Fisher-KPP
L'équation de Fisher-KPP, également connue sous le nom d'équation de Fisher-Kolmogorov-Petrovsky-Piskunov, est un cas particulier de système de réaction-diffusion. Elle porte le nom d'Andrey Kolmogorov, Ivan Petrovsky, Nikolai Piskunov et Ronald Fisher, et est utilisée pour modéliser la propagation de phénomènes comme l'expansion d'un allèle avantageux dans une population.

### 3. Détails sur l’équation de Fisher-KPP
L'équation de Fisher-KPP se présente sous la forme suivante :
$$
\partial_t u = D \nabla^2 u + ru(1 - u)
$$
où $u(x, t)$ représente la densité de population, $D$ est le coefficient de diffusion, et $r$ est le taux de croissance.

Cette équation admet des solutions en ondes progressives, décrivant le passage de l'état $u = 0$ (population nulle) à $u = 1$ (population stable). Pour une vitesse d'onde $c \geq 2\sqrt{rD}$, une solution en forme d'onde progressive existe :
$$
u(x, t) = v(x \pm ct) \equiv v(z)
$$
où $v$ est une fonction croissante satisfaisant $\lim_{{z \to -\infty}} v(z) = 0$ et $\lim_{{z \to \infty}} v(z) = 1$.

Les solutions d'onde progressive sont stables vis-à-vis des perturbations proches, mais peuvent être modifiées par des perturbations éloignées. De plus, la vitesse d'onde minimale permet de déterminer si une solution d'onde progressive existe ou non.